# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as sqal
from sklearn.model_selection import train_test_split

In [ ]:
# in_columns = 'message'
# cat_col_names = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [ ]:
# load data from database
engine = sqal.create_engine('sqlite:///../data/DisasterResponse.db')
# inspector = sqal.inspect(engine)
# inspector.get_schema_names()  # -> ['main']
# inspector.get_table_names(schema='main')  # -> ['Message']
# inspector.get_columns('Message', schema='main')   # -> the goods

df = pd.read_sql_table('MessageCategorization', engine)

In [ ]:
# correct outlying values in `related` column
df['related'] = np.clip(df['related'], 0, 1)

In [ ]:
df = df.drop('child_alone', axis=1)

In [ ]:
df.columns

In [ ]:
in_columns = 'message'
out_columns = list(df.columns)[4:]

In [ ]:
text = df[in_columns].values
y = df[out_columns].values

In [ ]:
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.33, random_state=42)

In [ ]:
y_train

In [ ]:
len(out_columns)

### 2. Write a tokenization function to process your text data

In [ ]:
import spacy
en_nlp = spacy.load('en')

In [ ]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
# https://realpython.com/natural-language-processing-spacy-python/ was helpful here
def tokenize(text):
    doc = en_nlp(text)
    lemmas = [token.lemma_ for token in doc if token not in stopwords and not token.is_punct]
    stems = [stemmer.stem(lemma) for lemma in lemmas]
    return stems

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

##### All the different classifiers I have tried:

In [ ]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import LinearSVC

In [ ]:
# model = make_pipeline(TfidfVectorizer(tokenizer=tokenize, min_df=5), 
#                      MultiOutputClassifier(estimator=
#                                            LinearSVC(C=1.0, dual=False, multi_class='ovr', fit_intercept=True, 
#                                                      max_iter=100)))

### 3b. Split pipeline into stages

In [ ]:
vect = TfidfVectorizer(tokenizer=tokenize, min_df=10, max_df=0.9)

In [ ]:
clf = MultiOutputClassifier(estimator=
                            LinearSVC(C=0.3, dual=False, multi_class='ovr', fit_intercept=True, 
                                      max_iter=100))

In [ ]:
model = make_pipeline(vect, clf)

In [ ]:
model.get_params

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

####  Let's split the `Vectorizer` out of the pipeline, precompute it on the training and test sets, and write the results to disk. 
Parameter searching is going to be a slow business and we want to speed things up as much as we can.

#### 4a. Train `vect` part of pipeline and cache results

In [ ]:
%%time
X_train = vect.fit_transform(text_train)

In [ ]:
%%time
X_test = vect.transform(text_test)

In [ ]:
# https://stackoverflow.com/questions/52444921/save-numpy-array-using-pickle
import pickle as pkl

In [ ]:
with open('severedPipelineXtrain.pkl', 'wb') as f:
    pkl.dump(X_train, f)

with open('severedPipelineXtest.pkl', 'wb') as f:
    pkl.dump(X_test, f)

#### 4b. Train `clf` part of pipeline

In [ ]:
%%time
clf.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
%%time
y_pred = clf.predict(X_test)

#### 5b. Generate basic classification report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, target_names=out_columns, zero_division=1))

#### 5c. Metrics by category

https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def genTable(true, pred, zero_division='warn'):
    table = np.empty(shape=(true.shape[1], 3))
    for i in range(true.shape[1]):
        table[i] = [precision_score(true[:,i], pred[:,i], zero_division=zero_division), \
                      recall_score(true[:,i], pred[:,i], zero_division=zero_division), \
                      f1_score(true[:,i], pred[:,i], zero_division=zero_division)]
    return table

def printScores(labels, true, pred, zero_division='warn'):
    results = genTable(true, pred, zero_division=zero_division)
    print("{0:>20}".format(''), 'prec', 'recall', 'f1', sep='\t')
    for i in range(results.shape[0]):
        print("{0:>20}".format(labels[i]), '%.2f' % results[i, 0], '%.2f' % results[i,1], '%.2f' % results[i,2], sep = '\t')

In [ ]:
printScores(out_columns, y_test, y_pred, zero_division=0)

### 6. Improve your model

#### Read back `X_train` and `X_test` from cache if necessary.

In [ ]:
# with open('severedPipelineXtrain.pkl', 'rb') as f:
#     X_train = pkl.load(f)

# with open('severedPipelineXtest.pkl', 'rb') as f:
#     X_test = pkl.load(f)


~~LR doesn't work here because of the empty 'child_alone' feature.~~  I've eliminated the 'child_alone' category because I want to use the LinearSVC estimator which cannot tolerate a category with only one class.

In [ ]:
print(clf.score(X_test, y_test))
print(clf.score(X_train, y_train))

Overfits like crazy with `RandomForestClassifier` (0.256, 0.995).  `AdaBoostClassifier` doesn't overfit at all, but it doesn't fit well either.  `LinearSVC` (0.291, 0.513) is somewhere in the middle, not least because it is remarkably fast.

With KNeighbors we get (0.233, 0.264) but it took 16 minutes to compute the two scores; but less than a second to fit.

In [ ]:
%%time
y_pred = clf.predict(X_test)

Took 5 mins with KNeighbors.  5 secs with RandomForest.  40 msecs with LinearSVC.

Wow!  RandomForest is a lot better than KNeighbors, but still awful.

In [ ]:
printScores(out_columns, y_test, y_pred, zero_division=0)

#### Let's do some parameter searching...

Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# param_grid = {}
# param_grid = {'multioutputclassifier__estimator__n_estimators': [10, 30, 100]} 
# param_grid = {'multioutputclassifier__estimator__learning_rate': [0.3, 1.0, 3.0]}
# param_grid = {'multioutputclassifier__estimator__C': [3.0, 10.0, 30.0]}
# param_grid = {'multioutputclassifier__estimator__l1_ratio': [0.1, 0.3, 0.7, 1.0]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_split': [2, 4, 8, 16]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': [1, 2, 4, 8]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__max_depth': [1, 2, 4, 8]}
# param_grid = {'multioutputclassifier__estimator__C': [0.3, 1.0, 3.0, 10.0]}
param_grid = {'estimator__C': [0.01, 0.03, 0.1, 0.3]}

In [ ]:
grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
grid.fit(X_train, y_train)

With Ada(RandomForest) fit() ran 53 minutes before I stopped it.

25min for miniscule benefit - LR with saga not worth pursuing.  (Although the highest score fit (l1_ratio = 0.7) was only 1/3 the duration of the most expensive.  Could be because of the warm_start setting.)

For the DecisionTreeClassifier, the default value of `min_samples_split` of 2 is the best, but the effect is very weak. [2,4,8,16]

Equilvalently, for DTC, the default value of `min_samples_leaf` was best, and varying it made no difference. [1,2,4,8]

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
grid.score(X_test, y_test)

In [ ]:
grid.cv_results_

DecisionTreeClassifier achieved (0.243, 0.238) with max_depth=2.  Grid search took 7 minutes.  Both min_samples_split and min_samples_leaf were negligable.

In [ ]:
# DecisionTreeClassifier base_estimator
parameters = {'min_samples_leaf': [1], 'min_samples_split': [2], 'max_depth': [2]}

# AdaBoostClassifier estimator: I also tried varying 'n_estimators' and 'learning_rate'.
# This had minimal impact so I chose speed over optimization.

cv = 0.243

#### 2nd Search - Search the `max_df` parameter of TfidfVectorizer

In [ ]:
param_grid = {'tfidfvectorizer__max_df': [0.05, 0.15, 0.5, 0.9]}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

In [ ]:
grid.fit(text_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.score(text_train, y_train)

In [ ]:
grid.score(text_test, y_test)

`max_df = 0.9` is the optimum from `param_grid={'tfidfvectorizer__max_df': [0.05, 0.15, 0.5, 0.9]}`, the effect is weak.  Also scores are essentially identical to a `max_df = 1.0` (the default).

In [ ]:
grid.best_estimator_.score(text_test, y_test)

In [ ]:
# searchMaxDF1 = grid.cv_results_
searchMaxDF1

#### 3rd Search - Search the `min_df` parameter of TfidfVectorizer

In [ ]:
param_grid = {'tfidfvectorizer__min_df': [0, 10, 100, 1000]}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
grid.fit(text_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_.score(text_test, y_test)

In [ ]:
grid.best_estimator_.score(text_train, y_train)

`min_df = 10` (0.304, 0.408) is the optimum from `param_grid={'tfidfvectorizer__min_df': [0, 10, 100, 1000]}`, the effect is weak.  Also scores are essentially identical to a `min_df = 5` (which is what I had been using).  But the much greater bonus is that it is substantially less overfit than `min_df = 5`.

In [ ]:
searchMinDF1 = grid.cv_results_
searchMinDF1

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
%%time
y_pred = model.predict(text_test)

In [ ]:
%%time
printScores(out_columns, y_test, y_pred, zero_division=0)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### See above.  I've already tested many things including various algorithms.

### 9. Export your model as a pickle file

In [ ]:
import pickle as pkl

In [ ]:
# Much niftier way of handling filehandles.
with open('classifier.pkl', 'wb') as f:
    pkl.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.